Note code is coming from Sebastian Raschka's [repo](https://github.com/rasbt/LLMs-from-scratch) covering LLMs from scratch with slight modifications

In [ ]:
import tiktoken
import torch
from torch.utils.data import DataLoader, Dataset
from utils import parse_sgm_to_dataframe

## Define dataloader

In [ ]:
def encode_and_decode_example(list_of_strings):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Get the token ID for <|endoftext|>
    endoftext_token = tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"})[0]

    all_tokens = []
    for text in list_of_strings:
        # Encode the text
        encoded = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
        all_tokens.extend(encoded + [endoftext_token])

    # Decode the tokens
    decoded = tokenizer.decode(all_tokens)

    return all_tokens, decoded

In [ ]:
class GPTDatasetV1(Dataset):
    def __init__(self, articles, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Get the token ID for <|endoftext|>
        endoftext_token = tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"})[0]

        # Tokenize all articles with end-of-text token
        all_tokens = []
        for article in articles:
            article_tokens = tokenizer.encode(article, allowed_special={"<|endoftext|>"})
            all_tokens.extend(article_tokens + [endoftext_token])

        # Use a sliding window to chunk the tokens into overlapping sequences of max_length
        for i in range(0, len(all_tokens) - max_length, stride):
            input_chunk = all_tokens[i:i + max_length]
            target_chunk = all_tokens[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def create_dataloader_v1(articles, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True, num_workers=0):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(articles, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader

## Load Reuters dataset

#### TODO
- Deduplicate articles so that there is 1 per ID, not 1 per topic
- Load all the articles, not just the single file

In [ ]:
df = parse_sgm_to_dataframe('../../data/reuters21578/reut2-000.sgm')

In [ ]:
articles = df['Body'].unique()

In [ ]:
len(articles)

## Create data pipeline

In [ ]:
vocab_size = 50257
output_dim = 256
max_len = 1024
context_length = max_len

In [ ]:
import torch
import torch.nn as nn

token_embedding_layer = nn.Embedding(vocab_size, output_dim)
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [ ]:
max_length = 4
dataloader = create_dataloader_v1(articles, batch_size=8, max_length=max_length, stride=max_length)

In [ ]:
def inspect_batch(x, y, n_samples=2):
    for i in range(min(n_samples, len(x))):
        tokenizer = tiktoken.get_encoding("gpt2")
        
        print(f"\nSample {i+1}:")
        
        # Decode and print the input sequence
        input_text = tokenizer.decode(x[i].tolist())
        print(f"Input text: {input_text}")
        print(f"Input encoding: {x[i].tolist()}")
        
        # Decode and print the target sequence
        target_text = tokenizer.decode(y[i].tolist())
        print(f"Target text: {target_text}")
        print(f"Target encoding: {y[i].tolist()}")
        
        print("-" * 50)

In [ ]:
INSPECT = True

In [ ]:
for batch in dataloader:
    x, y = batch

    if INSPECT:
        # Visual inspection
        inspect_batch(x, y)

    token_embeddings = token_embedding_layer(x)
    pos_embeddings = pos_embedding_layer(torch.arange(max_length))

    input_embeddings = token_embeddings + pos_embeddings

    break

In [ ]:
print(input_embeddings.shape)

# Define GPT model

In [ ]:
import torch.nn as nn

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by n_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)  # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.reshape(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift


class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))


class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        # Shortcut connection for attention block
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)   # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        # Shortcut connection for feed-forward block
        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut  # Add the original input back

        return x

In [ ]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits